# Jupyter notebook to compute metrics for performance comparison
## importing packages and building dataset path

In [1]:
from pathlib import Path
import lpips
import numpy
import glob
import torch
from skimage import metrics
from torchvision import transforms
from pytorch_msssim import ms_ssim
from PIL import Image

device = 'cuda' if torch.cuda.is_available() else 'cpu'

loss_fn_alex = lpips.LPIPS(net='alex')
loss_fn_vgg = lpips.LPIPS(net='vgg')

current_path = Path('.').resolve()
print(str(current_path))

reference_path = str(current_path)+'/IMAGES/PNG_IMAGES'
print("Reference source: ",reference_path)

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


C:\Users\filip\.conda\envs\imageCompressionAI\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\filip\.conda\envs\imageCompressionAI\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: C:\Users\filip\.conda\envs\imageCompressionAI\lib\site-packages\lpips\weights\v0.1\alex.pth
Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]


C:\Users\filip\.conda\envs\imageCompressionAI\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: C:\Users\filip\.conda\envs\imageCompressionAI\lib\site-packages\lpips\weights\v0.1\vgg.pth
C:\Users\filip\DataspellProjects\ImageCompressionAI
Reference source:  C:\Users\filip\DataspellProjects\ImageCompressionAI\IMAGES\PNG_IMAGES


## Preparing vector of folders to compare

In [2]:
datasets = {
    'JPEG':'/IMAGES/JPEG_IMAGES'
}
unused = {
    'JPEG2000':'/IMAGES/JPEG2000_IMAGES',
    'BPG':'/IMAGES/BPG_IMAGES',
    'VVC':'/IMAGES/VVC_IMAGES',
    'mbt2018_1':'/IMAGES/NET/mbt2018_1',
    'mbt2018_2':'/IMAGES/NET/mbt2018_2',
    'mbt2018_3':'/IMAGES/NET/mbt2018_3',
    'mbt2018_4':'/IMAGES/NET/mbt2018_4',
    'mbt2018_5':'/IMAGES/NET/mbt2018_5',
    'mbt2018_mean_1':'/IMAGES/NET/mbt2018_mean_1',
    'mbt2018_mean_2':'/IMAGES/NET/mbt2018_mean_2',
    'mbt2018_mean_3':'/IMAGES/NET/mbt2018_mean_3',
    'mbt2018_mean_4':'/IMAGES/NET/mbt2018_mean_4',
    'mbt2018_mean_5':'/IMAGES/NET/mbt2018_mean_5',
    'cheng2020_attn_1':'/IMAGES/NET/cheng2020_attn_1',
    'cheng2020_attn_2':'/IMAGES/NET/cheng2020_attn_2',
    'cheng2020_attn_3':'/IMAGES/NET/cheng2020_attn_3',
    'cheng2020_attn_4':'/IMAGES/NET/cheng2020_attn_4',
    'cheng2020_attn_5':'/IMAGES/NET/cheng2020_attn_5',
    'cheng2020-anchor_1':'/IMAGES/NET/cheng2020-anchor_1',
    'cheng2020-anchor_2':'/IMAGES/NET/cheng2020-anchor_2',
    'cheng2020-anchor_3':'/IMAGES/NET/cheng2020-anchor_3',
    'cheng2020-anchor_4':'/IMAGES/NET/cheng2020-anchor_4',
    'cheng2020-anchor_5':'/IMAGES/NET/cheng2020-anchor_5' 
}

## Computing metrics

In [3]:
for name, path in datasets.items():
    psnr_list=[]
    mssim_list=[]
    lpips_list=[]
    count = 0
    compressed_path = str(current_path)+path
    print('Evaluating '+name+' with dataset: '+str(compressed_path))
    for file in glob.glob(str(compressed_path)+'/*.png'):
        reference_image = Image.open(reference_path+'/'+Path(file).stem+'.png')
        reference = transforms.ToTensor()(reference_image).unsqueeze(0).to(device)
        compressed_image = Image.open(file)
        compressed = transforms.ToTensor()(compressed_image).unsqueeze(0).to(device)
        psnr = metrics.peak_signal_noise_ratio(numpy.array(reference_image),numpy.array(compressed_image),data_range=255)
        reference_image.close()
        compressed_image.close()
        mssim = ms_ssim(reference, compressed, data_range=1, size_average=False)
        d1 = loss_fn_alex(transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))(reference),transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))(compressed))
        d2 = loss_fn_vgg(transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))(reference),transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))(compressed))
        print(str(psnr) + ' ' + str(mssim.item()) + ' ' + str(d1[0].item()) + ' ' + str(d2[0].item()))
 
    

Evaluating JPEG with dataset: C:\Users\filip\DataspellProjects\ImageCompressionAI\IMAGES\JPEG_IMAGES
22.545904178905268 0.7591419219970703 0.6152011752128601 0.7280080318450928
23.753293333180103 0.658022403717041 0.6107895970344543 0.6072699427604675


KeyboardInterrupt: 